<a href="https://colab.research.google.com/github/Laura9704/Fundamentos_MCD/blob/main/Elecci%C3%B3n_Mejor_Modelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt #gráficos
import matplotlib.dates as mdates
pd.set_option('display.max_columns', None)

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import math
from math import sqrt

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dropout, Dense # Se usa SimpleRNN o la capa RNN que elijas
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Importa librerías para imputación
from scipy.interpolate import interp1d

import random
import os
import warnings
warnings.filterwarnings('ignore')

# Mejor Modelo MLP

# Mejor Modelo redes recurrentes

## Simple

En este modelo el que mejor representaciones tomo fue el modelo de capas apiladas, con los siguinetes caracteriticas y parametros.  

Dentro del modelado se configura un SimpleRNN para el entrenamiento de la siguiente forma

    model = Sequential([
        SimpleRNN(units,
                  activation='relu',
                  input_shape=(n_steps, n_features),
                  return_sequences=True),
        SimpleRNN(units // 2, activation='relu'),

        #SimpleRNN(units // 2, activation='relu', return_sequences=True),  # Segunda RNN, también devuelve secuencias

        #SimpleRNN(units // 4, activation='relu'),  # Tercera RNN, ahora sí sin return_sequences

        Dropout(drop_rate),
        Dense(32, activation='relu'),
        Dense(1, activation='linear')
    ])

siendo la arquietctura que mejor predijo en el entrenamiento, con los siguiente hiperparametros


    params = {'units': 128,
      'lr': 0.005,         
      'batch_size': 16,
      'epochs': 50,
      'drop_rate': 0.2,
      'huber_delta': 0.15,
      'loss_func': 'huber',
      'patience': 10  }

el cual al final se adapto para usar el codigo base quedando la SimpleRNN de esta forma

    class SimpleRNN_P(nn.Module):
      def __init__(self, input_size, hidden_size=128, drop_rate=0.2):
        super(SimpleRNN_P, self).__init__()
        
        # RNN 1: Salida es una secuencia (return_sequences=True implícito por el apilamiento)
        self.rnn1 = nn.RNN(input_size, hidden_size, batch_first=True, nonlinearity='relu')
        
        # RNN 2: Toma la secuencia de rnn1 y consolida (return_sequences=False implícito)
        self.rnn2 = nn.RNN(hidden_size, hidden_size // 2, batch_first=True, nonlinearity='relu')
        
        self.dropout = nn.Dropout(drop_rate)
        
        self.fc_seq = nn.Sequential(
            nn.Linear(hidden_size // 2, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        # x shape: (Batch, Seq_Len, Features)
        x_rnn1, _ = self.rnn1(x)  # x_rnn1 shape: (Batch, Seq_Len, Hidden_size)
        x_rnn2, _ = self.rnn2(x_rnn1) # x_rnn2 shape: (Batch, Seq_Len, Hidden_size // 2)
        
        # Tomamos la salida del último paso temporal para la predicción punto-a-punto
        x_out = x_rnn2[:, -1, :]
        
        x_out = self.dropout(x_out)
        
        # .squeeze(-1) asegura que la salida final sea (Batch_size,)
        return self.fc_seq(x_out).squeeze(-1)


## LTSM

## GRU

# Mejor Modelo TCN

#Eleccion Final